# Setting Up Mario

## Install Dependencies 


In [6]:
!pip install gym_super_mario_bros==7.3.0
!pip install nes-py 
!pip install pytouch

## Imports 

In [9]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

## Buidling the enviroment
Creates an enviroment and wraps the movment options into simple movment operations (256 movment operations down to 7)

In [26]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

## Running the Game
A loop to run the game with time-out length

In [24]:
done = True
for step in range(100000):
  if done:
    env.reset()
  state, reward, done, info = env.step(env.action_space.sample()) # Allows to pass through a movment operations to the game (Takes random actions at the moment)
  env.render()
env.close

KeyboardInterrupt: 

# Pre-Processing the Enviroment

## Installing AI Dependancies 

In [9]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


## Importing Wrappers for AI


In [11]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

In [27]:
# Create the base enviroment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

# Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# GrayScale the enviroment
env = GrayScaleObservation(env, keep_dim=True)

# Wrap inside the dummy enviroment
env = DummyVecEnv([lambda: env])

# Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

# Building the RI Model

In [28]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback



In [29]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [30]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [31]:
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [32]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log='./', learning_rate=0.000001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [19]:
model.learn(total_timesteps=500000, callback=callback)

Logging to ./PPO_5
----------------------------
| time/              |     |
|    fps             | 112 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 25            |
|    iterations           | 2             |
|    time_elapsed         | 40            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 3.1094998e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00106      |
|    learning_rate        | 1e-06         |
|    loss                 | 127           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000219     |
|    value_loss           | 326           |
------------------------------

KeyboardInterrupt: 

In [27]:
model.save('final')

# Testing it out


In [33]:
model = PPO.load('./train/best_model_500000')

In [34]:
state = env.reset()

while True:
    action, _state = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

KeyboardInterrupt: 

In [35]:
env.close()

In [ ]:
env.reset()